# Loading libraries, setting constants

In [14]:
from pathlib import Path
import numpy as np
import pandas as pd
import warnings
import os
import zipfile

## Extracting DATASET to Parent Folder (.SAS) ##
#### It will produce LLCP2022.XPT which is a .SAS file fomat

In [32]:
zip_file_path = Path('../')
zip_file_path2 = zip_file_path / 'LLCP2022XPT.zip'
extracted_dir_path = zip_file_path

with zipfile.ZipFile(zip_file_path2, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

In [15]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

DATA_PATH = Path('../')
RAW_FILE_PATH = DATA_PATH / 'LLCP2022.XPT'
PROCESSED_DATA_PATH = DATA_PATH / 'Post-DataProcessing'
PROCESSED_FILE_PATH_PARQUET = PROCESSED_DATA_PATH / 'heart_converted.parquet'

VAR_LIST_PATH = Path('./vars_list_with_descriptions.txt')

FINAL_FILE_NO_NANS_PATH = PROCESSED_DATA_PATH / 'heart_no_nans.csv'
FINAL_FILE_WITH_NANS_PATH = PROCESSED_DATA_PATH / 'heart_with_nans.csv'

## Converting SAS to PARQUET

In [8]:
def sas_converter(path: Path, dest_path: Path, file_format: str):
    assert file_format in ['csv', 'parquet'], "Invalid file format specified"
    
    try:
        # Read SAS file
        df = pd.read_sas(str(path), encoding='utf-8')

        # Create a copy of the DataFrame to reduce fragmentation
        df = df.copy()

        # Check if the destination directory exists, create it if it doesn't
        dest_path.parent.mkdir(parents=True, exist_ok=True)

        # Save to the specified format
        if file_format == 'csv':
            df.to_csv(dest_path, index=False)
        elif file_format == 'parquet':
            df.to_parquet(dest_path, index=False)
    except Exception as e:
        # Print the actual error
        print(f'Error: {e}')

# Call the function
sas_converter(path=RAW_FILE_PATH, dest_path=PROCESSED_FILE_PATH_PARQUET, file_format='parquet')

# Data preprocessing

## Reading, overviewing data

In [9]:
heart_df = pd.read_parquet(PROCESSED_FILE_PATH_PARQUET)
heart_df.info()
heart_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Columns: 326 entries, _STATE to _AIDTST4
dtypes: float64(321), object(5)
memory usage: 1.1+ GB


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_SMOKGRP,_LCSREC,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1.0,1.0,02032022,02,03,2022,1100.0,2022000001,2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,2.0,2.0
1,1.0,1.0,02042022,02,04,2022,1100.0,2022000002,2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,2.0,2.0,2.0
2,1.0,1.0,02022022,02,02,2022,1100.0,2022000003,2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,NaN,NaN,2.0
3,1.0,1.0,02032022,02,03,2022,1100.0,2022000004,2.022000e+09,1.0,...,3.0,2.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,9.0,9.0,2.0
4,1.0,1.0,02022022,02,02,2022,1100.0,2022000005,2.022000e+09,1.0,...,4.0,NaN,1.0,1.000000e+01,1.0,1.400000e+02,1.0,NaN,NaN,2.0


## Extracting the list of variables

In [10]:
NEW_VAR_NAMES = [
    "HeartDisease",
    "BMI",
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    "PhysicalHealth",
    "MentalHealth",
    "DiffWalking",
    "Sex",
    "AgeCategory",
    "Race",
    "Diabetic",
    "PhysicalActivity",
    "GenHealth",
    "SleepTime",
    "Asthma",
    "KidneyDisease",
    "SkinCancer"
]

In [16]:
var_list_df = pd.read_csv(VAR_LIST_PATH, sep=' - ', header=None, names=['Variable', 'Description'])

C:\Users\lenny\AppData\Local\Temp\ipykernel_8864\650844557.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  var_list_df = pd.read_csv(VAR_LIST_PATH, sep=' - ', header=None, names=['Variable', 'Description'])


In [17]:
var_list_df

,Variable,Description
0,CVDCRHD4,(Ever told) (you had) angina or coronary heart...
1,_BMI5,Body Mass Index (BMI)
2,SMOKE100,Have you ever smoked before?
3,DRNKANY6,Adults who reported having had at least one dr...
4,CVDSTRK3,(Ever told) (you had) a stroke.
5,PHYSHLTH,"Now thinking about your physical health, which..."
6,MENTHLTH,"Now thinking about your mental health, which i..."
7,DIFFWALK,Do you have serious difficulty walking or clim...
8,SEXVAR,Sex of Respondent
9,_AGEG5YR,Fourteen-level age category


In [18]:
var_list = var_list_df['Variable'].to_numpy()

In [19]:
var_list

array(['CVDCRHD4', '_BMI5', 'SMOKE100', 'DRNKANY6', 'CVDSTRK3',
       'PHYSHLTH', 'MENTHLTH', 'DIFFWALK', 'SEXVAR', '_AGEG5YR',
       '_RACEGR4', 'DIABETE4', 'EXERANY2', 'GENHLTH', 'SLEPTIM1',
       'ASTHMA3', 'CHCKDNY2', 'CHCSCNC1'], dtype=object)

In [20]:
heart_df = heart_df[var_list]

In [21]:
heart_df.columns = NEW_VAR_NAMES

In [22]:
SEX = {1: 'Male', 2: 'Female'}

GEN_HEALTH = {
    1: "Excellent",
    2: "Very good",
    3: "Good",
    4: "Fair",
    5: "Poor"
}

PHYS_MEN_HEALTH = {77: np.nan,
               88: 0,
               99: np.nan
                  }

YES_NO_QUESTIONS = {1: 'Yes', 2: 'No'}

SLEEP_TIME = lambda x: np.where(x > 24, np.nan, x)

DIABETES = {
    1: "Yes",
    2: "Yes, but only during pregnancy (female)",
    3: "No",
    4: "No, pre-diabetes or borderline diabetes",
}

SMOKER_STATUS = {
    1: "Current smoker - now smokes every day",
    2: "Current smoker - now smokes some days",
    3: "Former smoker",
    4: "Never smoked"
}

RACE = {
    1: "White only, Non-Hispanic",
    2: "Black only, Non-Hispanic",
    3: "Other race only, Non-Hispanic",
    4: "Multiracial, Non-Hispanic",
    5: "Hispanic"
}

AGE_CATEGORY = {
    1: "Age 18 to 24",
    2: "Age 25 to 29",
    3: "Age 30 to 34",
    4: "Age 35 to 39",
    5: "Age 40 to 44",
    6: "Age 45 to 49",
    7: "Age 50 to 54",
    8: "Age 55 to 59",
    9: "Age 60 to 64",
    10: "Age 65 to 69",
    11: "Age 70 to 74",
    12: "Age 75 to 79",
    13: "Age 80 or older"
}

In [23]:
heart_copy_df = heart_df.copy()

In [24]:
heart_copy_df['HeartDisease'] = heart_copy_df['HeartDisease'].map(YES_NO_QUESTIONS)
heart_copy_df['BMI'] = heart_copy_df['BMI'] / 100
heart_copy_df['Smoking'] = heart_copy_df['Smoking'].map(YES_NO_QUESTIONS)
heart_copy_df['AlcoholDrinking'] = heart_copy_df['AlcoholDrinking'].map(YES_NO_QUESTIONS)
heart_copy_df['Stroke'] = heart_copy_df['Stroke'].map(YES_NO_QUESTIONS)
heart_copy_df['PhysicalHealth'] = heart_copy_df['PhysicalHealth'].replace(PHYS_MEN_HEALTH)
heart_copy_df['MentalHealth'] = heart_copy_df['MentalHealth'].replace(PHYS_MEN_HEALTH)
heart_copy_df['DiffWalking'] = heart_copy_df['DiffWalking'].map(YES_NO_QUESTIONS)
heart_copy_df['Sex'] = heart_copy_df['Sex'].map(SEX)
heart_copy_df['AgeCategory'] = heart_copy_df['AgeCategory'].map(AGE_CATEGORY)
heart_copy_df['Race'] = heart_copy_df['Race'].map(RACE)
heart_copy_df['Diabetic'] = heart_copy_df['Diabetic'].map(DIABETES)
heart_copy_df['PhysicalActivity'] = heart_copy_df['PhysicalActivity'].map(YES_NO_QUESTIONS)
heart_copy_df['GenHealth'] = heart_copy_df['GenHealth'].map(GEN_HEALTH)
heart_copy_df['SleepTime'] = heart_copy_df['SleepTime'].apply(SLEEP_TIME)
heart_copy_df['Asthma'] = heart_copy_df['Asthma'].map(YES_NO_QUESTIONS)
heart_copy_df['KidneyDisease'] = heart_copy_df['KidneyDisease'].map(YES_NO_QUESTIONS)
heart_copy_df['SkinCancer'] = heart_copy_df['SkinCancer'].map(YES_NO_QUESTIONS)

In [25]:
def describe_df(df: pd.DataFrame):
    print(f"The dataset contains {df.shape[1]} columns and {len(df)} rows")

    for col in df.columns:
        col_dtype = df[col].dtype
        
        print(f"\nColumn: {col} ({col_dtype})")
        if col_dtype == 'object':
            print(f"--- Percentage of NaNs: {df[col].isna().sum() / len(df[col]) * 100}")
            print(f"--- Unique values:\n {df[col].unique()}")
        else:
            print(f"--- Summary statistics:\n {df[col].describe()}")

In [26]:
describe_df(heart_copy_df)

The dataset contains 18 columns and 445132 rows

Column: HeartDisease (object)
--- Percentage of NaNs: 0.9895940979305015
--- Unique values:
 ['No' 'Yes' nan]

Column: BMI (float64)
--- Summary statistics:
 count    396326.000000
mean         28.529842
std           6.554889
min          12.020000
25%          24.130000
50%          27.440000
75%          31.750000
max          99.640000
Name: BMI, dtype: float64

Column: Smoking (object)
--- Percentage of NaNs: 7.85385009390473
--- Unique values:
 ['No' 'Yes' nan]

Column: AlcoholDrinking (object)
--- Percentage of NaNs: 10.462963795009122
--- Unique values:
 ['No' 'Yes' nan]

Column: Stroke (object)
--- Percentage of NaNs: 0.34978388433094004
--- Unique values:
 ['No' 'Yes' nan]

Column: PhysicalHealth (float64)
--- Summary statistics:
 count    434205.000000
mean          4.347919
std           8.688912
min           0.000000
25%           0.000000
50%           0.000000
75%           3.000000
max          30.000000
Name: PhysicalHe

In [27]:
heart_copy_df.to_csv(FINAL_FILE_WITH_NANS_PATH, index=False)

In [28]:
heart_copy_df.dropna().to_csv(FINAL_FILE_NO_NANS_PATH, index=False)